#### Cyclistic : How Does a Bike-Share Navigate Speedy Success?

Initial R code source:
This analysis is for case study 1 from the Google Data Analytics Certificate (Cyclistic). It is originally based on the case study https://artscience.blog/home/divvy-dataviz-case-study written by Kevin Hartman. The R code has been adapted for use with updated data and file formats.



#### Install and load necessary packages

In [ ]:
install.packages('tidyverse')

In [ ]:
install.packages('lubridate')

In [ ]:
install.packages('hms')

In [ ]:
install.packages('data.table')

In [ ]:
install.packages("modeest")

In [ ]:
install.packages("openxlsx")


In [ ]:
library(dplyr)
library(ggplot2)
library(modeest)

In [ ]:
#load libraries 
library(tidyverse) #calculations
library(lubridate) #dates 
library(hms) #time
library(data.table)

In [ ]:
library(openxlsx)

#### Ask
###### Indentify the business task
The key business task in this case is to discover how casual riders and Cyclistic members use their rental bikes differently. Both the Director of Marketing as well as finance analysts have concluded that annual members are more profitable.

Therefore, the results of this analysis will be used to design a new marketing strategy to convert casual riders to annual members.

###### Consider key stakeholders.
Key stakeholders include: Cyclistic executive team, Director of Marketing (Lily Moreno), Marketing Analytics team.

#### Prepare
Download data and store it appropriately.
Data has been downloaded from Motivate International Inc. Local copies have been stored securely on Google Drive and here on Kaggle.

Identify how it’s organized.
All trip data is in comma-delimited (.CSV) format with 15 columns, including: ride ID #, ride type, start/end time, ride length (in minutes), day of the week, starting point (code, name, and latitude/longitude), ending point (code, name, and latitude/longitude), and member/casual rider.

Determine the credibility of the data.
Due to the fact that this is a case study using public data, we are going to assume the data is credible.

#### Process
Check the data for errors.
The code chunk below will import 12 individual .xlsx files as data frames, each representing 1 of the last 12 months of trip data. Some parsing errors persist, however, they represent less 0.25% of the data set, so this is still a representative sample.

In [ ]:
y2022_08 <- read.csv('/kaggle/input/case-study-cyclicist/case study/aug22.csv')

In [ ]:
y2022_09 <- read.csv('/kaggle/input/case-study-cyclicist/case study/sep22.csv')

In [ ]:
y2022_10 <- read.csv('/kaggle/input/case-study-cyclicist/case study/oct22.csv')

In [ ]:
y2022_11 <- read.csv('/kaggle/input/case-study-cyclicist/case study/nov22.csv')

In [ ]:
y2022_12 <- read.csv('/kaggle/input/case-study-cyclicist/case study/dec22.csv')

In [ ]:
y2023_01 <- read.csv('/kaggle/input/case-study-cyclicist/case study/jan23.csv')

In [ ]:
y2023_02 <- read.csv('/kaggle/input/case-study-cyclicist/case study/feb 23.csv')

In [ ]:
y2023_03 <- read.csv('/kaggle/input/case-study-cyclicist/case study/march23.csv')

In [ ]:
y2023_04 <- read.csv('/kaggle/input/case-study-cyclicist/case study/apr23.csv')

In [ ]:
y2023_05 <- read.csv('/kaggle/input/case-study-cyclicist/case study/may 23.csv')

In [ ]:
y2023_06 <- read.csv('/kaggle/input/case-study-cyclicist/case study/june23.csv')

In [ ]:
y2023_07<- read.csv('/kaggle/input/case-study-cyclicist/case study/july23.csv')

1 column was added to each of the 12 monthly .CSV files:

Day of the week (1 = Sunday, 7 = Saturday)


In [ ]:
#calculate the day of the week 
y2022_08 $day_of_week <- wday(y2022_08 $started_at)
y2022_09 $day_of_week <- wday(y2022_09 $started_at)
y2022_10 $day_of_week <- wday(y2022_10 $started_at)
y2022_11 $day_of_week <- wday(y2022_11 $started_at)
y2022_12 $day_of_week <- wday(y2022_12 $started_at)
y2023_01 $day_of_week <- wday(y2023_01 $started_at)
y2023_02 $day_of_week <- wday(y2023_02 $started_at)
y2023_03 $day_of_week <- wday(y2023_03 $started_at)
y2023_04 $day_of_week <- wday(y2023_04 $started_at)
y2023_05 $day_of_week <- wday(y2023_05 $started_at)
y2023_06 $day_of_week <- wday(y2023_06 $started_at)
y2023_07 $day_of_week <- wday(y2023_07 $started_at)



In [ ]:
head(y2022_08)

#### Document the cleaning process.


Day of the week (1 = Sunday, 7 = Saturday) column is added in to the all 12 
CSV files.


#### Analyze
Aggregating data so it’s useful and accessible.
This code chunk will combine the 12 individual data frames into one large data frame for analysis.

In [ ]:
all_trips_init <- bind_rows(
                       y2022_08, 
                       y2022_09, 
                       y2022_10, 
                       y2022_11, 
                       y2022_12, 
                       y2023_01, 
                       y2023_02, 
                       y2023_03, 
                       y2023_04, 
                       y2023_05, 
                       y2023_06, 
                       y2023_07, 
                       )

In [ ]:
head(all_trips_init)

Next, we add columns to list the date, month, day, and year of each ride for additional aggregation capabilities.

In [ ]:
## ----Add columns that list the date, month, day, and year of each ride for additional aggregation----
all_trips_init$date <- as.Date(all_trips_init$started_at)
all_trips_init$month <- format(as.Date(all_trips_init$date), "%m")
all_trips_init$day <- format(as.Date(all_trips_init$date), "%d")
all_trips_init$year <- format(as.Date(all_trips_init$date), "%Y")
all_trips_init$day_of_week <- format(as.Date(all_trips_init$date), "%A")

In [ ]:
head(all_trips_init)

We also remove any unnecessary columns (erroneous ride_length, latitude/longitude fields).

In [ ]:
all_trips_init <- all_trips_init %>% 
  select(-c( start_lat, start_lng, end_lat, end_lng))

We'll add another column to calculate each ride length in seconds.



In [ ]:
all_trips_init$ride_length <- as.numeric(difftime(all_trips_init$ended_at,all_trips_init$started_at))

Let's next remove the NA rows.



In [ ]:
all_trips_no_na <- drop_na(all_trips_init)

In [ ]:
null_count <- sum(sapply(all_trips_init, is.null))

In [ ]:
print(null_count)

The aggregated data frame includes approximately 10,500 entries (0.30% of total rides) when bikes were taken out of docks and checked for quality by Cyclistic or ride_length was negative. We will create a new version of the data frame since data is being removed.

In [ ]:
## ----Remove negative ride length and quality check rows---------------------
all_trips <- all_trips_no_na[!(all_trips_no_na$start_station_name == "HQ QR" | all_trips_no_na$ride_length<0),]

In [ ]:
str(all_trips)


Perform calculations.
Let's first look at a statistical summary of the aggregated and transformed data frame. Let's also look at the structure of the columns.

In [ ]:
summary(all_trips)
str(all_trips)

Let's next focus on the average length of each ride, this time in minutes. We see that on average, each ride is close to 30 minutes. We'll then break that down by casual riders versus members.

In [ ]:
summary(all_trips$ride_length)/60

Looking at casual riders versus members, we can see that the average casual ride is about 27.7 minutes compared to the members' average ride of 12.3 minutes. The median rides are 12 minutes and 8 minutes respectively.

In [ ]:
aggregate(all_trips$ride_length/60 ~ all_trips$member_casual, FUN = mean)
aggregate(all_trips$ride_length/60 ~ all_trips$member_casual, FUN = median)

In [ ]:
aggregate(all_trips$day_of_week ~ all_trips$member_casual, FUN = mfv)

Also, we can take a look at the average ride time by day for members and casual riders with duration again in minutes. Regardless of day of the week, casual users ride 2.7x to 3x longer than members, with both groups riding longer on weekends.

In [ ]:
all_trips$day_of_week <- ordered(all_trips$day_of_week, levels=c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"))
aggregate(all_trips$ride_length/60 ~ all_trips$member_casual + all_trips$day_of_week, FUN = mean)

In [ ]:
all_trips %>%
  mutate(weekday_num = lubridate::wday(started_at)) %>%
  mutate(weekday_label = case_when(
    weekday_num == 1 ~ "Sun",
    weekday_num == 2 ~ "Mon",
    weekday_num == 3 ~ "Tue",
    weekday_num == 4 ~ "Wed",
    weekday_num == 5 ~ "Thu",
    weekday_num == 6 ~ "Fri",
    weekday_num == 7 ~ "Sat",
    TRUE ~ NA_character_
  )) %>%
  group_by(member_casual, weekday_label) %>%
  summarise(number_of_rides = n(), average_duration = mean(ride_length/60)) %>%
  arrange(member_casual, weekday_label)

##### Identify trends and relationships.
Here are some key observations using the simple analyses above:

Casual riders average longer rides than members. This makes sense as members are likely using the bike rental service for particular commutes (i.e. work or school) whereas casual riders are just that--casual (perhaps they are sightseeing)


The number of rides for both types of users starts off slow on Mondays, gradually increases to a peak on Saturdays with a small drop off on Sundays.

#### Share






In [ ]:
## ----Number of rides by rider type------------------------------------------
all_trips %>%
  mutate(weekday = factor(wday(started_at), 
                          levels = 1:7,
                          labels = c("Sun", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat"))) %>%
  group_by(member_casual, weekday) %>%
  summarise(number_of_rides = n(),
            average_duration = mean(ride_length)) %>%
  arrange(member_casual, weekday) %>%
  ggplot(aes(x = weekday, y = number_of_rides, fill = member_casual)) +
  geom_col(position = "dodge") +
  labs(title = "Table 1: Number of Rides by Day and Rider Type",
       y = "Number of Rides (1e+05 = 100,000)",
       x = "Day of Week")

Saturdays emerged as the day with the highest ride count for casual riders, whereas wednesday claimed the record for member riders’ most frequent day of riding.

In [ ]:
## ----Average duration-------------------------------------------------------
all_trips %>% 
  mutate(weekday = factor(wday(started_at), 
                          levels = 1:7,
                          labels = c("Sun", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat"))) %>%
  group_by(member_casual, weekday) %>% 
  summarise(number_of_rides = n()
            ,average_duration = mean(ride_length/60)) %>% 
  arrange(member_casual, weekday)  %>% 
  ggplot(aes(x = weekday, y = average_duration, fill = member_casual)) +
  geom_col(position = "dodge") + 
  labs(title = "Table 2: Average Ride Duration by Day and Rider Type") + 
  ylab("Average Duration (minutes)") + 
  xlab("Day of Week")

There was a notable difference of six hundred thousand in the count of rides between registered member riders and casual riders. Interestingly, despite this, casual riders surpassed member riders in terms of ride duration, spending over twice the amount of time on their rides. Furthermore, the weekly average for maximum ride length among casual riders significantly exceeded that of member riders.

In [ ]:
## ----Number of rides by day and bike type-----------------------------------
all_trips %>% 
  mutate(weekday = factor(wday(started_at), 
                          levels = 1:7,
                          labels = c("Sun", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat"))) %>% 
  group_by(rideable_type, weekday) %>% 
  summarise(number_of_rides = n()
            ,average_duration = mean(ride_length)) %>% 
  arrange(rideable_type, weekday)  %>% 
  ggplot(aes(x = weekday, y = number_of_rides, fill = rideable_type)) +
  geom_col(position = "dodge") + 
  labs(title = "Table 4: Number of Rides by Day and Bike Type") + 
  ylab("Number of Rides (1e+05 = 100,000)") + 
  xlab("Day of Week")

The electric bike type has a considerably higher number of rides when compared to both the classic bike and the docked bike, which has a significantly lower ride count. 

Throughout the recorded duration, the classic bike stood out as the most prevalent type of rideable. Interestingly, both member riders and casual riders favored the classic bike for their rides. Notably, member riders opted not to utilize the docked bike at any point during this period.

In [ ]:
## ----Number of rides by month and rider type--------------------------------
all_trips %>% 
  group_by(member_casual, month) %>% 
  summarise(number_of_rides = n()
            ,average_duration = mean(ride_length)) %>% 
  arrange(member_casual, month)  %>% 
  ggplot(aes(x = month, y = number_of_rides, group = member_casual)) +
  geom_line(aes(color = member_casual)) + 
  geom_point() +
  labs(title = "Table 6: Number of Rides by Month and Rider Type") + 
  ylab("Number of Rides (1e+05 = 100,000)") + 
  xlab("Month")

In terms of peak riding activity, casual riders reached their highest point in July, whereas member riders observed their peak during August. Conversely, both categories of riders experienced their lowest activity of the year in January.

### Here is a summary of the key observations from above:

Casual riders have notably longer average ride durations compared to members.

Saturdays are the preferred riding day for casual riders, while Wednesdays are popular among members.

Casual riders tend to ride more on weekends, whereas members have consistent riding patterns throughout the week.

On any given day, casual riders' rides are significantly longer than those of members.

Despite 600,000 more rides by casual riders, they spend more than twice the time on their rides compared to members.

Electric bikes are the most popular, followed by classic bikes, while docked bikes have fewer rides.

Classic bikes are favored by both member and casual riders, with no member rides on docked bikes.

Peak riding activity occurs in July for casual riders and in August for members, with low activity in January.

### Act
#### top three recommendations based on your analysis
Weekend-Only Membership: Consider introducing a weekend-only membership option at a different price point than the full annual membership. This targeted membership would allow casual users to unlock bikes exclusively on Fridays, Saturdays, and Sundays. This strategy aligns with casual riders' weekend-oriented riding patterns, potentially enticing them to transition to a full annual membership for more flexibility during the week.

"See Our City" Campaign: Launch a "See Our City" campaign specifically aimed at casual users. This campaign could include 52 suggested bike routes covering all major sights in the city, with one route for each weekend of the year. By renting bikes for these curated routes, casual riders could explore the city's attractions over the course of a year while also enjoying cost savings compared to individual rentals. This campaign could encourage frequent weekend riders to commit to a full annual membership.

Summer Marketing Emphasis: Ensure that marketing campaigns are timed to coincide with the peak riding activity, which is in the summer. Consider launching promotional offers, discounts, or special events during the summer months to attract both casual riders and members. Highlight the benefits of riding during this season, such as pleasant weather and longer daylight hours, to encourage increased ridership and membership conversions.

These recommendations leverage the observed differences in riding behavior between casual users and members to tailor marketing strategies that could enhance user engagement, loyalty, and membership conversion.